# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [12]:
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory

import os
import joblib
import pandas as pd
import sys

from sklearn.model_selection import train_test_split




## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-automl'

experiment=Experiment(ws, experiment_name)



In [16]:
datastores = ws.datastores
for name, datastore in datastores.items():
    print(name, datastore.datastore_type)

datastore AzureBlob
workspacefilestore AzureFile
workspaceblobstore AzureBlob


In [31]:
df = Dataset.get_by_name(ws, name='corona-nlp-train').to_pandas_dataframe()

In [39]:
for col in df.columns:
     if "Col" in col:
         df = df.drop(col)

KeyError: "['Column7'] not found in axis"

In [40]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,None,None,None,None,None,None,None,None,None
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,None,None,None,None,None,None,None,None,None
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,None,None,None,None,None,None,None,None,None
3,3802,48754,None,16-03-2020,My food stock is not the only one which is emp...,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [41]:
df.Sentiment.unique()

array(['Neutral', 'Positive', None, 'Negative', 'Extremely Positive',
       'Extremely Negative',
       ' please save them from this COVID-19. Please sustain them and let them feel your comfort and love ??"',
       'Bread', ' staff our hospitals', ' be #healthy &amp; #immune',
       ' wash your hands. https://t.co/GFQIk6Gr5x"',
       ' the panic may wane even more."', ' Woodstock sanctuary',
       ' those who have lost their jobs due to all the shutdowns"',
       ' of course', ' no flour. At least In 1 supermarket',
       ' etc please contact Cheney Brothers at 478-250-3699. Office: 352-291-7800',
       ' potatoes &amp; vegetables??', ' or grocery store',
       ' &amp; others who are literally out there.', ' consumed more',
       ' teachers', ' milk powder gone""',
       ' ""We\'ll beÂ\x85 https://t.co/e49zUcJV70"',
       ' teachers and grocery store attendants.',
       ' die from hunger.854 million people worldwide are estimated to be undernourished',
       ' firefighte

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [35]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes":30,
    "task":"classification",
    "primary_metric":"accuracy",
    "training_data":df,
    "label_column_name":"Sentiment",
    "n_cross_validations":3}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

In [36]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_c0d8ecb6-3730-4773-9a56-51abe6e28a56

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lea

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run_automl, best_model_automl = automl_run.get_output()
best_run_metrics = best_run_automl.get_metrics()
parameter_values = best_run_automl.get_details()

print("Best Run Id: ", best_run_automl.id)
print("\n")
print("\n")
print("Accuracy: ", best_run_metrics["accuracy"])
print("\n")
print("\n")
print("Parameters: ", parameter_values)
print("\n")
print("\n")
print("Metrics for best run: ", best_run_automl.get_metrics())

In [ ]:
# referencce - https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.run.automlrun?view=azure-ml-py
print(best_model_automl.steps[-1])

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Personal ToDo - Check why can't register model from get_output() directly
best_model_automl = best_run_automl.register_model(model_name="capstone-automl.pkl", model_path="./")

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service